In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import scipy.integrate
import astropy.units as u
import GCRCatalogs
import pandas as pd

GCRCatalogs.get_available_catalog_names(include_default_only=False)

['base5000_v1.1.1',
 'base5000_v1.1.1_9431_9812',
 'base5000_v1.1.1_9556',
 'base5000_v1.1.1_image',
 'base5000_v1.1.1_small',
 'base5000_v1.1.1_z_0_1',
 'base5000_v1.1.1_z_1_2',
 'baseDC2_snapshot_z0.15_v0.1',
 'baseDC2_snapshot_z0.15_v0.1_small',
 'baseDC2_snapshot_z1.01_v0.1',
 'baseDC2_v1.1_9431_9812',
 'baseDC2_v1.1_9556',
 'baseDC2_v1.1_image',
 'buzzard',
 'buzzard_high-res',
 'buzzard_high-res_v1.1',
 'buzzard_test',
 'buzzard_v2.0.0_3',
 'buzzard_v2.0.0_4',
 'buzzard_v2.0.0_test',
 'cosmoDC2',
 'cosmoDC2_v1.0',
 'cosmoDC2_v1.0_10194_10452',
 'cosmoDC2_v1.0_8786_9049',
 'cosmoDC2_v1.0_9050_9430',
 'cosmoDC2_v1.0_9431_9812',
 'cosmoDC2_v1.0_9556',
 'cosmoDC2_v1.0_9813_10193',
 'cosmoDC2_v1.0_image',
 'cosmoDC2_v1.0_image_addon_knots',
 'cosmoDC2_v1.0_small',
 'cosmoDC2_v1.1.4',
 'cosmoDC2_v1.1.4_image',
 'cosmoDC2_v1.1.4_image_addon_knots',
 'cosmoDC2_v1.1.4_image_combined_agn',
 'cosmoDC2_v1.1.4_image_with_photoz_calib',
 'cosmoDC2_v1.1.4_image_with_photozs_flexzboost_v1',
 'co

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import scipy.integrate
import astropy.units as u
import GCRCatalogs
import pandas as pd

%matplotlib inline

try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

import sys
import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

catalog = 'skysim5000_v1.1.1'
skysim_cat = GCRCatalogs.load_catalog(catalog)
cosmo_ss  = skysim_cat.cosmology



from astropy.io import fits
from astropy.table import Table

ncdata_fits = fits.open('skysim5000_halos_m200c_13.fits')
#ncdata_fits.info()


ncdata_data =  ncdata_fits[1].data
print(ncdata_fits[1].columns)


ncdata_Table = Table(ncdata_data)
ncdata_Table
ncdata_Table['M200c'] = ncdata_Table['baseDC2/sod_halo_mass']/cosmo_ss.h # sod_halo_mass = M200,c in Msun/h, needs conversion
N_cl = len(ncdata_Table['halo_mass'])
'''
mmin_extract = 1.e12 # Msun (M_fof)
dm_halos = skysim_cat.get_quantities(['halo_mass','redshift','baseDC2/sod_halo_mass', 'baseDC2/redshift','baseDC2/sod_halo_cdelta'],
                                     filters=[f'halo_mass > {mmin_extract}','is_central==True',
                                              f'redshift_true>0.0', f'redshift_true<1.5'])
N_cl = len(dm_halos['halo_mass'])
'''
print(f'There are {N_cl} halos in this mass (Mfof) and redshift range in an area of {skysim_cat.sky_area}')

ColDefs(
    name = 'dec_true'; format = 'D'
    name = 'ra_true'; format = 'D'
    name = 'redshift_true'; format = 'D'
    name = 'baseDC2/target_halo_redshift'; format = 'D'
    name = 'halo_mass'; format = 'D'; disp = 'G13.4'
    name = 'baseDC2/target_halo_fof_halo_id'; format = 'K'
    name = 'baseDC2/sod_halo_mass'; format = 'D'; disp = 'G13.4'
    name = 'halo_id'; format = 'K'
    name = 'baseDC2/sod_halo_radius'; format = 'D'
)
There are 5357544 halos in this mass (Mfof) and redshift range in an area of 5264.05


In [3]:
def mcmc(zmin,zmax,mmin,mult):
    mmax = 1.e15
    from astropy.io import fits
    from astropy.table import Table



    #dm_halos['M200c'] = dm_halos['baseDC2/sod_halo_mass']/cosmo_ss.h # sod_halo_mass = M200,c in Msun/h, needs conversion

    filt3 = ncdata_Table['M200c'] >= mmin
    filt4 = ncdata_Table['M200c'] < mmax
    filt5 = ncdata_Table['redshift_true'] >= zmin
    filt6 = ncdata_Table['redshift_true'] < zmax

    filt = filt3 * filt4 * filt5 * filt6

    data_m_ss = ncdata_Table['M200c'][filt] # M200,c [Msun]
    data_z_ss = ncdata_Table['redshift_true'][filt]
    N_cl = len(data_m_ss)
    print(f'There are {N_cl} halos in this mass (Msod) and redshift range')
    
    #cosmological model
    cosmo = Nc.HICosmoDEXcdm()
    reion = Nc.HIReionCamb.new () 
    prim = Nc.HIPrimPowerLaw.new () 

    cosmo.add_submodel (reion)
    cosmo.add_submodel (prim)

    dist = Nc.Distance.new (2.0)

    tf = Nc.TransferFunc.new_from_name ("NcTransferFuncEH")

    psml = Nc.PowspecMLTransfer.new (tf)

    #psml = Nc.PowspecMLCBE.new ()
    psml.require_kmin (1.0e-6)
    psml.require_kmax (1.0e3)

    psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
    psf.set_best_lnr0 ()
    cosmo.props.H0      = cosmo_ss.H0.value
    cosmo.props.Omegab  = cosmo_ss.Ob0
    cosmo.props.Omegac  = cosmo_ss.Odm0
    cosmo.props.Omegax  = cosmo_ss.Ode0

    cosmo.omega_x2omega_k ()
    cosmo.param_set_by_name ("Omegak", 0.0)

    prim.props.n_SA = cosmo_ss.n_s
    print(cosmo_ss.sigma8,cosmo.sigma8(psf),cosmo.Omega_k0())

    old_amplitude = math.exp (prim.props.ln10e10ASA)
    prim.props.ln10e10ASA = math.log ((cosmo_ss.sigma8 / cosmo.sigma8(psf))**2 * old_amplitude)
    print(cosmo_ss.sigma8, cosmo.sigma8(psf))
    
    #mass and multiplicty functions
    if mult == 'Tinker':
        mulf = Nc.MultiplicityFuncTinker.new_full (Nc.MultiplicityFuncMassDef.CRITICAL,200)
    if mult == 'Bocquet':
        mulf = Nc.MultiplicityFuncBocquet.new_full (Nc.MultiplicityFuncMassDef.CRITICAL,0,200)
    if mult != 'Tinker' and mult != 'Bocquet' : 
        raise TypeError('only Tinker or Bocquet')
    mf = Nc.HaloMassFunction.new (dist, psf, mulf)
    
    #proxies
    lnM_min = math.log (mmin)
    lnM_max = math.log (mmax)
    cluster_m = Nc.ClusterMass.new_from_name ("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (lnM_min, lnM_max))
    cluster_z = Nc.ClusterRedshiftNodist(z_min = zmin, z_max=zmax)
    
    #dataset
    cad = Nc.ClusterAbundance.new (mf, None)
    ncdata = Nc.DataClusterNCount.new (cad, "NcClusterRedshiftNodist", "NcClusterMassNodist")
    mset = Ncm.MSet.new_array ([cosmo, cluster_z, cluster_m])
    
    ncdata.set_lnM_true(Ncm.Vector.new_array(np.log (data_m_ss)))
    ncdata.set_z_true(Ncm.Vector.new_array(data_z_ss))
    ncdata.set_lnM_obs (Ncm.Matrix.new_array (data_m_ss,1))
    ncdata.set_z_obs (Ncm.Matrix.new_array (data_z_ss,1))
    mf.set_area_sd (skysim_cat.sky_area)
    ncdata.true_data (True)
    ncdata.set_init(True)
    cosmo.props.Omegac_fit    = True
    cosmo.props.Omegab_fit    = False
    prim.props.ln10e10ASA_fit = True

    dset = Ncm.Dataset.new ()
    dset.append_data (ncdata)

    lh = Ncm.Likelihood.new (dset)
    #fit
    
    fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_FORWARD)
    m2lnL_bf = fit.m2lnL_val()
    fit.log_info ()
    fit.obs_fisher ()
    fit.log_covar ()
    
    cosmo.props.Omegac  = cosmo_ss.Odm0
    prim.props.ln10e10ASA = math.log ((cosmo_ss.sigma8 / cosmo.sigma8(psf))**2 * old_amplitude)
    m2lnL_f = fit.m2lnL_val()
    likeratio = m2lnL_f/m2lnL_bf
    
    
    #mcmc
    Ncm.func_eval_set_max_threads (150)
    Ncm.func_eval_log_pool_stats ()
    mfunc_oa = Ncm.ObjArray.new ()

    mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
    mfunc_Omegam = Ncm.MSetFuncList.new ("NcHICosmo:Omega_m0", None)

    mfunc_oa.add (mfunc_sigma8)
    mfunc_oa.add (mfunc_Omegam)

    print (mfunc_sigma8.eval0 (mset))
    print (mfunc_Omegam.eval0 (mset))
    init_sampler = Ncm.MSetTransKernGauss.new (0)
    init_sampler.set_mset (mset)
    init_sampler.set_prior_from_mset ()
    init_sampler.set_cov_from_rescale (1.0) #1
    sampler = 'apes'
    nwalkers = int (math.ceil (500)) #500
    ssize    = 15000 #1000000

    if sampler == 'apes':
      walker = Ncm.FitESMCMCWalkerAPES.new (nwalkers, mset.fparams_len ())
    elif sampler == "stretch":
      walker = Ncm.FitESMCMCWalkerStretch.new (nwalkers, mset.fparams_len ())
    esmcmc  = Ncm.FitESMCMC.new_funcs_array (fit, nwalkers, init_sampler, walker, Ncm.FitRunMsgs.SIMPLE, mfunc_oa)
    esmcmc.set_data_file ("z_%.1f_%.1f_M_%.e_%.e_%s.fits" % (zmin, zmax,mmin,mmax,mult))
    esmcmc.set_nthreads (150)
    esmcmc.start_run ()
    esmcmc.run (ssize / nwalkers)
    esmcmc.end_run ()
    esmcmc.mean_covar ()
    fit.log_covar ()
    
    print('# case z_%.1f_%.1f_M_%.e_%.e_%s  -2lnL_Bestfit = %f , -2lnL_Fiducial = %f , then  -2lnL_Fiducial/-2lnL_Bestfit = %f' % (zmin, zmax, mmin, mmax, mult, m2lnL_bf, m2lnL_f,likeratio))    

In [4]:
mcmc(0.0,1.0,3e14,'Tinker')
mcmc(0.0,1.0,3e14,'Bocquet')

There are 4277 halos in this mass (Msod) and redshift range
0.8 0.8490796381180891 0.0
0.8 0.8
#----------------------------------------------------------------------------------
# Data used:
#   - Cluster abundance unbinned
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  71                  [FIXED]
#   -     Omegac[01]:  0.22                [FREE]
#   -     Omegak[02]:  0                   [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0448              [FIXED]
#   -          w[07]: -1                   [FIXED]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law 


(process:16947): NUMCOSMO-WARNING **: 17:19:47.917: ncm_fit_fisher_to_covar: covariance matrix not positive definite, errors are not trustworthy.


# NcmThreadPool:Unused:      1
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      150
0.8490796381180891
0.2648
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensamble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0150.
#   Using MPI:         no - use MPI enabled but no slaves available.
# NcmFitESMCMC: No RNG was defined, using algorithm: `mt19937' and seed: 394097333.
# NcmMSetCatalog: Current mean:   63874        0.98036      0.26443      0.21963      3.3315     
# NcmMSetCatalog: Current msd:    3710         0.016957     0.00043086   0.00043086   0.032748   
# NcmMSetCatalog: Current sd:     82959        0.37917      0.0096342    0.0096342    0.73227    
# NcmMSetCatalog: Current var:    6.8822e+09   0.14377      9.2818e-05   9.2818e-05   0.53622    
# NcmMSetCatalog: Current tau:    1            1            1 

In [5]:
mcmc(0.0,1.0,4e14,'Tinker')
mcmc(0.0,1.0,4e14,'Bocquet')

There are 1810 halos in this mass (Msod) and redshift range
0.8 0.8490796381180891 0.0
0.8 0.8
#----------------------------------------------------------------------------------
# Data used:
#   - Cluster abundance unbinned
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  71                  [FIXED]
#   -     Omegac[01]:  0.22                [FREE]
#   -     Omegak[02]:  0                   [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0448              [FIXED]
#   -          w[07]: -1                   [FIXED]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law 


(process:16947): NUMCOSMO-WARNING **: 17:35:10.340: ncm_fit_fisher_to_covar: covariance matrix not positive definite, errors are not trustworthy.


# NcmThreadPool:Unused:      1
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      150
0.8490796381180891
0.2648
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensamble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0150.
#   Using MPI:         no - use MPI enabled but no slaves available.
# NcmFitESMCMC: No RNG was defined, using algorithm: `mt19937' and seed: 67040342.
# NcmMSetCatalog: Current mean:   38839        0.97934      0.2649       0.2201       3.3268     
# NcmMSetCatalog: Current msd:    2353.9       0.016727     0.0004249    0.0004249    0.033051   
# NcmMSetCatalog: Current sd:     52634        0.37403      0.0095011    0.0095011    0.73904    
# NcmMSetCatalog: Current var:    2.7704e+09   0.1399       9.0271e-05   9.0271e-05   0.54618    
# NcmMSetCatalog: Current tau:    1            1            1  

In [6]:
mcmc(0.0,0.5,4e14,'Tinker')
mcmc(0.0,0.5,4e14,'Bocquet')

There are 841 halos in this mass (Msod) and redshift range
0.8 0.8490796381180891 0.0
0.8 0.8
#----------------------------------------------------------------------------------
# Data used:
#   - Cluster abundance unbinned
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  71                  [FIXED]
#   -     Omegac[01]:  0.22                [FREE]
#   -     Omegak[02]:  0                   [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0448              [FIXED]
#   -          w[07]: -1                   [FIXED]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law m


(process:16947): NUMCOSMO-WARNING **: 17:53:08.562: ncm_fit_fisher_to_covar: covariance matrix not positive definite, errors are not trustworthy.


# NcmThreadPool:Unused:      1
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      150
0.8490796381180891
0.2648
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensamble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0150.
#   Using MPI:         no - use MPI enabled but no slaves available.
# NcmFitESMCMC: No RNG was defined, using algorithm: `mt19937' and seed: 72624697.
# NcmMSetCatalog: Current mean:   6065.9       0.98243      0.26388      0.21908      3.336      
# NcmMSetCatalog: Current msd:    319.76       0.017101     0.00046473   0.00046473   0.033599   
# NcmMSetCatalog: Current sd:     7150.1       0.38238      0.010392     0.010392     0.75129    
# NcmMSetCatalog: Current var:    5.1124e+07   0.14622      0.00010799   0.00010799   0.56443    
# NcmMSetCatalog: Current tau:    1            1            1  

In [7]:
mcmc(0.5,1.0,4e14,'Tinker')
mcmc(0.5,1.0,4e14,'Bocquet')

There are 969 halos in this mass (Msod) and redshift range
0.8 0.8490796381180891 0.0
0.8 0.8
#----------------------------------------------------------------------------------
# Data used:
#   - Cluster abundance unbinned
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  71                  [FIXED]
#   -     Omegac[01]:  0.22                [FREE]
#   -     Omegak[02]:  0                   [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0448              [FIXED]
#   -          w[07]: -1                   [FIXED]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law m


(process:16947): NUMCOSMO-WARNING **: 18:17:42.627: ncm_fit_fisher_to_covar: covariance matrix not positive definite, errors are not trustworthy.


# NcmThreadPool:Unused:      1
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      150
0.8490796381180891
0.2648
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensamble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0150.
#   Using MPI:         no - use MPI enabled but no slaves available.
# NcmFitESMCMC: No RNG was defined, using algorithm: `mt19937' and seed: 2171109758.
# NcmMSetCatalog: Current mean:   31310        0.96546      0.26472      0.21992      3.299      
# NcmMSetCatalog: Current msd:    2024.5       0.016675     0.00044851   0.00044851   0.032872   
# NcmMSetCatalog: Current sd:     45270        0.37287      0.010029     0.010029     0.73505    
# NcmMSetCatalog: Current var:    2.0493e+09   0.13903      0.00010058   0.00010058   0.5403     
# NcmMSetCatalog: Current tau:    1            1            1

In [8]:
mcmc(0.5,1.0,3e14,'Tinker')
mcmc(0.5,1.0,3e14,'Bocquet')

There are 2500 halos in this mass (Msod) and redshift range
0.8 0.8490796381180891 0.0
0.8 0.8
#----------------------------------------------------------------------------------
# Data used:
#   - Cluster abundance unbinned
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  71                  [FIXED]
#   -     Omegac[01]:  0.22                [FREE]
#   -     Omegak[02]:  0                   [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0448              [FIXED]
#   -          w[07]: -1                   [FIXED]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law 


(process:16947): NUMCOSMO-WARNING **: 18:39:40.177: ncm_fit_fisher_to_covar: covariance matrix not positive definite, errors are not trustworthy.


# NcmThreadPool:Unused:      1
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      150
0.8490796381180891
0.2648
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensamble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0150.
#   Using MPI:         no - use MPI enabled but no slaves available.
# NcmFitESMCMC: No RNG was defined, using algorithm: `mt19937' and seed: 4141663750.
# NcmMSetCatalog: Current mean:   51040        0.97056      0.26458      0.21978      3.3219     
# NcmMSetCatalog: Current msd:    3060.1       0.016024     0.00042159   0.00042159   0.031831   
# NcmMSetCatalog: Current sd:     68425        0.3583       0.009427     0.009427     0.71177    
# NcmMSetCatalog: Current var:    4.682e+09    0.12838      8.8869e-05   8.8869e-05   0.50662    
# NcmMSetCatalog: Current tau:    1            1            1